# Neural Network GRU

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.signal import iirfilter, filtfilt
from sklearn.preprocessing import StandardScaler
import dill
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

"""
Date: 2024-06-20 15:47:56,,,,,,
Calibration: XL_ODR: 6667Hz, XL_FS: 16g, GY_ODR: 6667Hz, GY_FS: 2000dps,,,
Time(s),Acceleration X (g),Acceleration Y (g),Acceleration Z (g),Angular Momentum X (dps),Angular Momentum Y (dps),Angular Momentum Z (dps)
0.000083,0.693359,-0.720215,0.14209,-3.540039,0.366211,-3.051758
0.00098,0.70752,-0.755371,0.086914,1.953125,0.732422,-1.159668
"""

# Check if GPU is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Helper module to trim convolution output to the original sequence length.
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, : -self.chomp_size].contiguous()


# A single Temporal Block (residual block) including two dilated convolution layers.
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1, self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


# The TCN model composed of multiple Temporal Blocks.
class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        """
        num_inputs: Number of features per time step.
        num_channels: List of output channels for each Temporal Block.
        """
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2**i
            in_channels = num_inputs if i == 0 else num_channels[i - 1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size, padding=(kernel_size - 1) * dilation_size, dropout=dropout)]
        self.network = nn.Sequential(*layers)
        self.fc = nn.Linear(num_channels[-1], 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x shape: (batch, seq_len, features)
        x = x.transpose(1, 2)  # (batch, features, seq_len)
        y = self.network(x)
        y = y[:, :, -1]  # take the output of the final time step
        y = self.fc(y)
        y = self.sigmoid(y)
        return y
    
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, seq_len, step):
        self.data = data
        self.labels = labels
        self.seq_len = seq_len
        self.step = step

    def __len__(self):
        return len(self.data) - (self.seq_len - 1) * self.step

    def __getitem__(self, idx):
        indices = [idx + i * self.step for i in range(self.seq_len)]
        x_seq = self.data[indices]
        y_val = self.labels[indices[-1]]
        return x_seq, y_val


In [ ]:
# Load training data
train_data = pd.read_csv(r"../measurement/processed_data/kanguru_train.csv", header=2)

window_size = 10
averaged_rows = []

for i in range(0, len(train_data), window_size):
    chunk = train_data.iloc[i : i + window_size]
    chunk_mean = chunk.mean(numeric_only=True)
    # Use mean time for the new row
    chunk_mean["Time(s)"] = chunk["Time(s)"].mean()
    averaged_rows.append(chunk_mean)

train_data = pd.DataFrame(averaged_rows, columns=train_data.columns)

X_train = train_data[["Time(s)", "Acceleration X (g)", "Acceleration Y (g)", "Acceleration Z (g)", "Angular Momentum X (dps)", "Angular Momentum Y (dps)", "Angular Momentum Z (dps)"]]
y_train = train_data["Stand detected"]

# Compute sampling frequency and filter parameters
sampling_frequency = 1 / (train_data["Time(s)"].diff().mean())
cutoff_frequency = 100
b, a = iirfilter(4, Wn=cutoff_frequency, fs=sampling_frequency, btype="low", ftype="butter")

# Apply filter, standardize and weight features
X_train_filtered = X_train.copy()
X_train_filtered.iloc[:, 1:] = filtfilt(b, a, X_train.iloc[:, 1:], axis=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filtered.iloc[:, 1:])
feature_weights = np.array([1, 1, 1, 1, 1, 1])
X_train_weighted = (X_train_scaled * feature_weights).astype(np.float32)

# Convert to tensor
X_train_tensor = torch.tensor(X_train_weighted)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

# Reintroduce the custom TimeSeriesDataset with sequence length and step
sequence_length = 10
step = 10
batch_size = 512

In [ ]:
train_dataset = TimeSeriesDataset(X_train_tensor, y_train_tensor, sequence_length, step)
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define TCN model instance
num_features = X_train_tensor.shape[1]
model = TCN(num_inputs=num_features, num_channels=[32, 32], kernel_size=2, dropout=0.2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 25
model.train()
for epoch in range(num_epochs):
    with tqdm(train_loader, unit="batch") as tepoch:
        for batch_X, batch_y in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            # TCN expects input shape (batch, seq_len, features)
            outputs = model(batch_X)
            loss = criterion(outputs.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
            tepoch.set_postfix(loss=loss.item())

print("Training complete.")

torch.save(model.state_dict(), "model/tcn_kang_model.pth")

In [ ]:
model = TCN(num_inputs=X_train_tensor.shape[1], num_channels=[32, 32], kernel_size=2, dropout=0.2).to(device)
model.load_state_dict(torch.load('model/tcn_kang_model.pth'))

# Load test data
test_data = pd.read_csv(r"../measurement/processed_data/kanguru_test.csv", header=2)

averaged_rows = []

for i in range(0, len(test_data), window_size):
    chunk = test_data.iloc[i : i + window_size]
    chunk_mean = chunk.mean(numeric_only=True)
    # Use mean time for the new row
    chunk_mean["Time(s)"] = chunk["Time(s)"].mean()
    averaged_rows.append(chunk_mean)

test_data = pd.DataFrame(averaged_rows, columns=test_data.columns)

X_test = test_data[["Time(s)", "Acceleration X (g)", "Acceleration Y (g)", "Acceleration Z (g)", "Angular Momentum X (dps)", "Angular Momentum Y (dps)", "Angular Momentum Z (dps)"]]

# Apply IIR filter to test data (excluding time series)
X_test_filtered = X_test.copy()
X_test_filtered.iloc[:, 1:] = filtfilt(b, a, X_test.iloc[:, 1:], axis=0)

# Standardize the test data
X_test_scaled = scaler.transform(X_test_filtered.iloc[:, 1:])

# Convert test data to PyTorch tensor
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# Prepare test dataset with sequence windows and change to batch prediction
test_dataset = TimeSeriesDataset(X_test_tensor, torch.zeros(len(X_test_tensor)), sequence_length, 10)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

all_preds = []
model.eval()
with torch.no_grad():
    for batch_X, _ in test_loader:
        batch_X = batch_X.to(device)  # shape: (batch, seq_len, features)
        batch_preds = model(batch_X).squeeze() > 0.5
        all_preds.extend(batch_preds.tolist())

# Align predictions with the original test indices.

pred_array = np.full(len(X_test_tensor), np.nan)
for i, pred in enumerate(all_preds):
    index = i + (sequence_length - 1) * step
    if index < len(pred_array):
        pred_array[index] = pred

test_data["predicted_heel_button"] = pred_array

In [ ]:
# Plot Angular Momentum Z axis (filtered and scaled)
fig = go.Figure()
# [:, 5]
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 0], mode="lines", name="AC X (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 1], mode="lines", name="AC Y (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 5], mode="lines", name="AM Z (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=test_data["Stand detected"], mode="lines", name="Heel Button"))
#fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=test_data["Stand detected"], mode="lines", name="Analytical"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=test_data["predicted_heel_button"], mode="lines", name="Predicted Heel Button"))

fig.show()

In [ ]:
test_data["Time(s)"] = test_data["Time(s)"] - test_data.iloc[0]["Time(s)"]
test_data.to_csv("../measurement/processed_data/comparison_kang/tcn_test.csv")

test_data = test_data[test_data["Time(s)"] > 0.3]
test_data["Time(s)"] = test_data["Time(s)"] - test_data.iloc[0]["Time(s)"]

equal_values = (test_data["predicted_heel_button"] == test_data["Stand detected"]).sum()
print(f"{equal_values} out of {test_data.shape[0]} entries are equal, which is {equal_values / test_data.shape[0] * 100:.2f}%")

def count_artifacts(pred_signal, time_vector, threshold_0=0.05, threshold_1=0.15):
    """
    Counts artifacts in a binary step signal based on duration thresholds.
    
    Args:
        pred_signal (np.array): Array of predicted values (0 or 1).
        time_vector (np.array): Array of corresponding time values.
        threshold_0 (float): Minimum duration in seconds for a valid 0 segment.
        threshold_1 (float): Minimum duration in seconds for a valid 1 segment.
        
    Returns:
        Tuple (count_zero, count_one): Number of artifact segments for 0 and 1.
    """
    n = len(pred_signal)
    count_zero = 0
    count_one = 0
    start_idx = 0

    while start_idx < n:
        current = pred_signal[start_idx]
        end_idx = start_idx + 1
        while end_idx < n and pred_signal[end_idx] == current:
            end_idx += 1

        # Compute segment duration
        duration = time_vector[end_idx - 1] - time_vector[start_idx]

        if current == 0 and duration < threshold_0:
            count_zero += 1
        elif current == 1 and duration < threshold_1:
            count_one += 1

        start_idx = end_idx

    return count_zero, count_one

count_zero, count_one = count_artifacts(test_data["predicted_heel_button"].values, test_data["Time(s)"].values)
print(f"Number of short 0 segments (artifacts): {count_zero}")
print(f"Number of short 1 segments (artifacts): {count_one}")

print(f" {equal_values / test_data.shape[0] * 100:.2f}; {count_zero}; {count_one};")